# Crop Images

***

**Author:** Shane Cooke

**Date:** 26 Feb 2023

**Description:** 

****
#### Imports

In [2]:
import json
import os
from PIL import Image
import copy
import numpy as np
import statistics

***
#### Default JSON Components

In [23]:
new_json_file = {
        "info": {},
        "licenses": [],
        "categories": [
            {
                "id": 1,
                "name": "Overfilled"
            },
            {
                "id": 2,
                "name": "NonOverfilled"
            }
        ],
        "images": [],
        "annotations": []
    }

default_images = {
            "file_name": "Hello.jpg",
            "id": 0,
            "width": 0,
            "height": 0
        }    

default_annotations = {
            "image_id": 0,
            "id": 0,
            "category_id": 1,
            "area": 0,
            "iscrowd": 0,
            "bbox": [0, 0, 0, 0],
            "segmentation": []
        }

***
#### Crop Images & Create New JSON

In [24]:
ds = 'Train'

image_path = f'./Dataset/HRI/Reduced/{ds}/'
f = open(f'Dataset/HRI/Reduced/split_train.json')
data = json.load(f)
f.close()

In [26]:
index = 0
save_path = './Dataset/HRI/Reduced CROP/'


# Iterating through the json
for count, a in enumerate(data["annotations"]):
    # Initilise Image & Annotations Details
    image_details = default_images
    annotation_details = default_annotations

    # Get Details from JSON File
    id = a["image_id"]
    category = a["category_id"]
    area = a['area']
    iscrowd = a['iscrowd']
    segmentation = a['segmentation']
    for i in data["images"]:
        if (id == i["id"]):
            filename = i["file_name"]
    bbox = a["bbox"]

    # Crop Image
    image = Image.open(image_path + filename)
    cropped = image.crop((bbox[0], bbox[1], (bbox[0]+bbox[2]), (bbox[1]+bbox[3])))

    # Save Cropped Image
    check = os.path.isfile(f"{save_path}{ds}/{filename}")
    if check == False:
        saved_name = filename
        cropped.save(f'{save_path}{ds}/{saved_name}')
    else:
        fn = filename.replace('.jpg', '')
        saved_name = f'{fn}_{index}.jpg'
        cropped.save(f'{save_path}{ds}/{saved_name}')
        index = index + 1

    # Populate Image & Annotations Details
    image_details['file_name'] = saved_name
    image_details['id'] = count
    image_details['width'] = cropped.size[0]
    image_details['height'] = cropped.size[1]
    annotation_details['image_id'] = count
    annotation_details['id'] = count
    annotation_details['category_id'] = category
    annotation_details['area'] = area
    annotation_details['iscrowd'] = iscrowd
    annotation_details['bbox'][2] = a['bbox'][2]
    annotation_details['bbox'][3] = a['bbox'][3]
    annotation_details['segmentation'] = segmentation

    # Append Details into JSON
    new_json_file['images'].append(copy.deepcopy(image_details))
    new_json_file['annotations'].append(copy.deepcopy(annotation_details))

In [27]:
j_file = json.dumps(new_json_file, indent=4)
with open(f"{save_path}split_train_crop.json", 'w') as outfile:
    outfile.write(j_file)

***
# Resize Images

In [5]:
height = 480
width = 704

input_image_path = './Dataset/HRI/Control/Val/'
output_image_path = './Dataset/HRI/Reduced/Val'
os.makedirs(output_image_path,exist_ok=True)

In [6]:
for image_name in os.listdir(input_image_path):

    image_path = os.path.join(input_image_path, image_name)

    if os.path.isfile(image_path):
        image = Image.open(image_path).convert('RGB')
        image = image.resize((width, height))

        image = image.save(f"{output_image_path}/{image_name}")

In [17]:
f = open(f'Dataset/Personal/Control/split_val.json')
data = json.load(f)
f.close()

In [18]:
index = 0
# Iterating through the json
for a in data['images']:
    a['width'] = 704
    a['height'] = 480

for a in data['annotations']:
    a['bbox'][0] = (a['bbox'][0] * 704/6016)
    a['bbox'][1] = (a['bbox'][1] * 480/4000)
    a['bbox'][2] = (a['bbox'][2] * 704/6016)
    a['bbox'][3] = (a['bbox'][3] * 480/4000)

In [19]:
j_file = json.dumps(data, indent=4)
with open(f"Dataset/Personal/SizedDown/split_val.json", 'w') as outfile:
    outfile.write(j_file)

***

# Find Size for Cropped Images

In [3]:
f = open('./Dataset/HRI/Reduced CROP/split_train_crop.json')
data = json.load(f)
f.close()

In [4]:
ratio_array = []
size_array = []
for image in data['images']:
    ratio = image['height'] / image['width']
    size = [image['width'], image['height']]
    ratio = round(ratio, 2)
    ratio_array.append(ratio)
    size_array.append(size)

ratio_array = sorted(ratio_array)
size_array = sorted(size_array)

print(statistics.median(ratio_array))

1.415


In [5]:
first_elements = [sublist[0] for sublist in size_array]
median_first_element = statistics.median(first_elements)
print(median_first_element)

162.0


In [6]:
median_first_element * statistics.median(ratio_array)

229.23000000000002